# Lesson 04: Concurrency & Parallelism

## 1. Parallelism

Compared to concurrency, parallelism is easier to use, and is _usually_ easier to think about and design.

We can explore this through a data-processing scenario: going through a very large CSV/JSON doc and filtering out columns or keys.
This is usually done when selecting a relevant subset of data from a very broad data set, usually sourced from a 3rd party.

e.g. You want to download the wikipedia dataset and filter for actors, movie titles and release years, so that you can make a simple and comprehensive list.

e.g.2 You want to make a demo example for this lesson, so you have to fake the data before demonstrating the filtering

### Example 1: Generating a text file

1. We need to generate a _very_ large NDJSON file (newline-delimited JSON). For simplicities sake, all lines are readable/same schema etc.

    ```json
    {"a": "B"}
    {"a": "C"}
    ```

1.1. There are many language and OS-level optimisations around doing the _exact_ same thing, like performing the same calculation over the same file line data. This means that we have to randomise the values in order to make a good test file.

    > use `faker`

In [1]:
from faker import (Faker, providers)
F = Faker()
F.add_provider(providers.misc)
F.add_provider(providers.geo)

In [2]:
def fkr_n(fkr, n): return [fkr() for _ in range(n)]

In [3]:
def gen_movie(f=F):
    return {
        "titleId":         f.uuid4(),
        "ordering":        f.random_int(),
        "title":           f.catch_phrase(),
        "region":          f.locale(),
        "language":        f.language_name(),
        "types":           fkr_n(f.name, 5),
        "attributes":      fkr_n(f.name, 5),
        "isOriginalTitle": f.boolean(),
        "tconst":          f.uuid4(),
        "titleType":       f.domain_name(),
        "primaryTitle":    f.catch_phrase(),
        "originalTitle":   ":".join([f.company(), f.catch_phrase()]),
        "isAdult":         f.boolean(),
        "startYear":       f.date(),
        "endYear":         f.year(),
        "runtimeMinutes":  f.random_int(),
        "genres":          fkr_n(f.country, 5),
        "tconst":          f.hex_color(),
        "directors":       fkr_n(f.name, 2),
        "writers":         fkr_n(f.name, 15),
        "actors":          fkr_n(f.name, 50),
    }

In [4]:
import json
from IPython.display import JSON
JSON(gen_movie(F))

<IPython.core.display.JSON object>

Woohoo! Now we just need to write this to a file

Lets make a function that loops and `yields` data

In [5]:
class MovieTable:
    def records(fpath, n_records=10):
        print(f"Writing {n_records} records to {fpath}")
        with open(fpath, "w") as ostream:
            for line in MovieTable.iter(n_records):
                print(line, file=ostream)

    def iter(n_records=10):
        for _ in range(n_records):
            yield gen_movie()

In [6]:
%timeit MovieTable.records("/tmp/movies.ndjson", 20)

Writing 20 records to /tmp/movies.ndjson
Writing 20 records to /tmp/movies.ndjson
Writing 20 records to /tmp/movies.ndjson
Writing 20 records to /tmp/movies.ndjson
Writing 20 records to /tmp/movies.ndjson
Writing 20 records to /tmp/movies.ndjson
Writing 20 records to /tmp/movies.ndjson
Writing 20 records to /tmp/movies.ndjson
213 ms ± 6.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


But I need to show you the CPU usage _per core_!

What about threading?

> Use psutil

In [7]:
import psutil
def cpu():
    print("\t".join(map(str, psutil.cpu_percent(percpu=True))))

In [8]:
import threading
import time

def loop_with_timer():
    timer = threading.Timer(1, cpu, args=None, kwargs=None)
    timer.start()

    # Actually do stuff here
    for i in range(5):
        print(i)
        time.sleep(1)

    timer.cancel()

loop_with_timer()

0
0.9	0.9	1.9	0.9	1.9	0.9
1
2
3
4


Huh? The threading. Timer only printed once?!

- The "interval" of the timer is more like a "delay"   
- from the docs:
    ```
    "Create a timer that will run function with arguments args and keyword arguments kwargs, 
    after interval seconds have passed."
    ```

In [9]:
# Shout out -> https://stackoverflow.com/a/48741004
class RepeatTimer(threading.Timer):
    def run(self):
        while not self.finished.wait(self.interval):
            self.function(*self.args, **self.kwargs)

def loop_with_timer():
    timer = RepeatTimer(2, cpu, args=None, kwargs=None)
    timer.start()

    for i in range(5):
        print(i)
        time.sleep(1)

    timer.cancel()

loop_with_timer()

0
1
1.3	1.2	1.5	0.5	1.3	1.5
2
3
2.5	2.0	3.5	6.1	3.0	1.0
4


Hmmm, there's a downside here -   
In every function that we want to track CPU usage for, we have to add all of this threading code -_-

However - we could make a decorator function that would allow anything to be timed just by adding a @decoration

In [10]:
import functools

def with_cpu(func):
    'Looks gross, but you only have to write it once!'

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        timer = RepeatTimer(1, cpu, args=None, kwargs=None)
        timer.start()

        result = func(*args, **kwargs)

        timer.cancel()
        print("fin!")
        return result
    return wrapper

Now, we can just add `@with_cpu` to any function that we need to investigate

In [11]:
@with_cpu
def poc():
    for i in range(5):
        print(i)
        time.sleep(1)

In [12]:
poc()

0
1
5.8	6.2	4.3	4.3	5.8	3.3
2
0.0	2.0	1.0	1.0	4.0	3.0
0.0	0.0	1.0	0.0	0.0	0.0
3
1.0	1.0	2.0	2.0	4.1	0.0
4
fin!
1.0	0.0	1.0	0.0	1.0	0.0


Let's add the decorator to our `MovieTable.records` method

In [13]:
import math
class MovieTable:
    @with_cpu
    def records(fpath, n_records=10):
        print(f"Writing {n_records} records to {fpath}")
        with open(fpath, "w") as ostream:
            for i, line in enumerate(MovieTable.iter(n_records)):
                print(line, file=ostream)

    def iter(n_records=10):
        for _ in range(n_records):
            yield gen_movie()

In [14]:
MovieTable.records("/tmp/movies.ndjson", 500)

Writing 500 records to /tmp/movies.ndjson
95.3	6.5	1.9	2.8	4.7	3.7
100.0	0.0	0.0	0.0	0.0	0.0
100.0	3.9	1.9	1.0	0.0	1.0
100.0	1.0	1.0	0.0	1.0	0.0
100.0	3.0	2.0	2.0	0.0	1.0
fin!


Nice!

- Now we're running some code, an running another looping bit of code in a thread on the side.
- We can see that we are currently using 1 core

- what about `async`?

In [15]:
import json
import asyncio
from dataclasses import dataclass
import functools

import psutil

@dataclass
class Timer:
    f: object
    sentinel: bool = False

    def task(self):
        async def run():
            while not self.sentinel:
                self.f()
                await asyncio.sleep(1)
        return asyncio.create_task(run())

    async def stop(self):
        self.sentinel = True

async def run_with_timer(f: functools.partial, t: Timer):
    tsk = t.task()
    await f()
    tsk.cancel()

    try:
        await tsk
    except asyncio.CancelledError:
        print("finished")

In [16]:
class AMovieTable:
    async def records(fpath, n_records=10):
        print(f"Writing {n_records} records to {fpath}")
        with open(fpath, "w") as ostream:
            for _ in range(n_records):
                print(json.dumps(gen_movie()), file=ostream, end="\n")
                await asyncio.sleep(0)

In [24]:
import asyncio

def run_async(f):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # 'RuntimeError: There is no current event loop...'
        loop = None

    if loop and loop.is_running():
        task = loop.create_task(f)
        task.add_done_callback(lambda _: print('fin!'))
    else:
        asyncio.run(f)

run_async(
    run_with_timer(functools.partial(AMovieTable.records, *("/tmp/movies.ndjson",500)), Timer(cpu))
)

Writing 500 records to /tmp/movies.ndjson
0.8	1.0	0.6	5.9	3.3	3.3
0.0	0.0	0.0	0.0	0.0	0.0
2.0	1.0	2.0	6.8	1.0	100.0
0.0	0.0	0.0	0.0	0.0	0.0
0.0	1.0	1.0	2.9	2.0	100.0
0.0	0.0	0.0	0.0	0.0	0.0
1.0	2.0	2.0	5.0	1.0	100.0
0.0	0.0	0.0	0.0	0.0	100.0
0.0	0.0	0.0	1.0	1.0	100.0
0.0	0.0	0.0	0.0	0.0	100.0
0.0	1.0	2.0	5.9	1.0	100.0
0.0	0.0	0.0	0.0	0.0	100.0
fin!
finished
fin!


Cool, cool. How could we do the same thing, but with a decorator?

In [25]:

def awith_cpu(func):
    @functools.wraps(func)
    async def wrapped(*args, **kwargs):
        tsk = Timer(cpu).task()
        result = await func(*args, **kwargs)
        tsk.cancel()
        try:
            await tsk
        except asyncio.CancelledError:
            print("fin!")
            return result
    return wrapped

In [27]:
class AMovieTable:
    @awith_cpu
    async def records(fpath, n_records=10):
        print(f"Writing {n_records} records to {fpath}")
        with open(fpath, "w") as ostream:
            for _ in range(n_records):
                print(json.dumps(gen_movie()), file=ostream, end="\n")
                await asyncio.sleep(0)

import asyncio

def run_async(f):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # 'RuntimeError: There is no current event loop...'
        loop = None

    if loop and loop.is_running():
        task = loop.create_task(f)
        task.add_done_callback(lambda _: print('double fin!'))
    else:
        asyncio.run(f)
run_async(AMovieTable.records("/tmp/movies.ndjson",500))

Writing 500 records to /tmp/movies.ndjson
1.9	3.3	5.7	3.2	2.7	1.4
13.7	0.0	2.0	1.0	86.4	4.0
100.0	0.0	0.0	0.0	1.0	1.0
100.0	2.0	1.0	1.0	4.9	1.9
100.0	1.9	0.0	1.0	1.0	0.0
100.0	0.0	1.0	2.9	6.9	2.0
fin!
double fin!


In [20]:
!wc -l "/tmp/movies.ndjson" && ls -alh "/tmp/movies.ndjson"

Writing 500 records to /tmp/movies.ndjson
4.2	4.3	16.0	0.0	4.2	95.8
4 /tmp/movies.ndjson
-rw-r--r-- 1 jovyan users 23K Nov 30 11:14 /tmp/movies.ndjson


In [21]:
from multiprocessing import Pool, Process

procs = []
for fpath in ["/tmp/movies.ndjson", "/tmp/movies2.ndjson"]:
    p = Process(target=MovieTable.records, args=(fpath,500))
    p.start()
    procs.append(p)

while True:
    if all(p.is_alive() for p in procs):
        print("fin!")
        break

fin!
Writing 500 records to /tmp/movies.ndjson
Writing 500 records to /tmp/movies2.ndjson
6.3	62.6	8.5	82.3	7.0	72.26.9	62.8	9.7	82.4	6.9	72.3

99.0	0.0	1.0	100.0	4.0	99.0
99.0	1.0	2.9	100.0	3.8	99.0
100.0	3.9	3.0	99.0	7.0	100.0
100.0	3.9	3.0	99.0	7.9	100.0
100.0	2.0	0.0	100.0	7.8	99.0
100.0	1.9	1.0	100.0	6.9	99.0
100.0	3.9	3.0	100.0	7.1	100.0
99.0	4.9	2.0	100.0	8.1	100.0
fin!
fin!


In [22]:
def run_async_pool(args):
    run_async(AMovieTable.records, args)

def _():
    with Pool(2) as p:
        p.map(run_async_pool, [("/tmp/movies.ndjson", 500), ("/tmp/movies2.ndjson", 500)])

In [23]:
# _()

8.3	29.3	9.9	67.1	4.8	48.2
10.0	100.0	25.0	100.0	25.0	100.0
35.7	65.1	3.7	100.0	4.8	98.8
100.0	0.0	0.0	100.0	0.0	100.0
100.0	2.3	0.0	98.8	6.9	100.0
100.0	10.5	15.0	100.0	12.5	100.0
100.0	2.4	0.0	100.0	9.4	98.8
100.0	0.0	0.0	100.0	0.0	100.0
100.0	2.3	1.2	100.0	4.8	100.0
100.0	10.5	10.5	100.0	16.7	100.0
98.9	2.4	1.2	98.9	6.0	100.0
100.0	0.0	0.0	100.0	0.0	95.2
45.9	2.3	1.2	43.5	4.7	100.0
0.0	11.1	5.6	9.5	5.3	100.0
0.0	1.1	0.0	0.0	1.1	100.0
0.0	0.0	5.0	0.0	0.0	100.0
1.1	2.3	0.0	0.0	2.3	100.0
0.0	0.0	5.6	5.9	0.0	100.0
0.0	1.1	0.0	0.0	2.3	100.0
0.0	5.0	0.0	0.0	0.0	100.0
4.7	1.2	1.2	2.3	1.2	100.0
4.3	8.3	4.5	0.0	0.0	100.0
2.5	1.2	0.0	53.1	0.0	43.0
finished
fin!
fin!
fin!
